In [3]:
from src.methods import (
    FullFineTuner,
    LoRAFineTuner,
    LoRAFAFineTuner,
    LoRAPlusFineTuner,
    DeltaLoRAFineTuner,
    AdaLoRAFineTuner,
    QLoRAFineTuner,
    VeRAFineTuner,
    PTuningV2FineTuner,
    PromptTuningFineTuner
)

In [ ]:
full = FullFineTuner()
full.run(save_model=True)

In [ ]:
lora = LoRAFineTuner()
lora.run(save_model=True)

In [ ]:
lora_fa = LoRAFAFineTuner()
lora_fa.run(save_model=True)

In [ ]:
lora_plus = LoRAPlusFineTuner()
lora_plus.run(save_model=True)

In [ ]:
delta_lora = DeltaLoRAFineTuner()
delta_lora.run(save_model=True)

In [ ]:
ada_lora = AdaLoRAFineTuner()
ada_lora.run(save_model=True)

In [ ]:
qlora = QLoRAFineTuner()
qlora.run(save_model=True)

In [ ]:
vera = VeRAFineTuner()
vera.run(save_model=True)

In [ ]:
ptuningv2 = PTuningV2FineTuner()
ptuningv2.run(save_model=True)

In [ ]:
prompt_tuning = PromptTuningFineTuner()
prompt_tuning.run(save_model=True)